In [1]:
import pickle
import sys

import pandas as pd
sys.path.insert(0, '/Users/lhao/Documents/Git/py_Halo-GEM/ecpy')

import dfba_functions
import numpy as np

In [2]:
def test_growth(model):
    model.objective = 'Biomass_v1'
    model.objective_direction = 'max'
    s = model.optimize()
    r = s.objective_value if s.status == 'optimal' else -1
    print('Results:')
    print('  Status        :',s.status)
    print('  growth_rate   :',r)
    print('  glucose uptake:',s.fluxes['Exchange_Glucopyranose'])
    print('  PHA           :',s.fluxes['PHA_secretion'])
    print('  NGAM lb:',model.reactions.NGAM.lower_bound)
    print('  PHA lb:',model.reactions.PHA_secretion.lower_bound)
    
    print()
            

def do_dfba_defalut(model):
    biomass_id = 'Biomass_v1'
    glc_id = 'Exchange_Glucopyranose'
    d_model = dfba_functions.dfba_model('test_model')
    d_model.tspan = np.linspace(0, 50, 101)
    d_model.cobra_model = model.copy()

    d_model.products_rxns = [biomass_id, 'PHA_secretion',
                             'Exchange_ACET']  # 'PHA_secretion', 'Exchange_ACET', 'Exchange_PYRUVATE',
    d_model.substrate_rxns = [glc_id]
    d_model.substrate_bounds = ['upper_bound']
    d_model.opt_list = d_model.products_rxns + d_model.substrate_rxns  # [biomass_id, 'EX_co2_e', 'EX_ac_e', 'EX_pyr_e', 'EX_glc__D_e']
    d_model.opt_list_direction = ['max', ] * (len(d_model.opt_list) - 1) + ['min']
    d_model.opt_list_multiply_by = np.array([1] * (len(d_model.opt_list) - 1) + [-1])
    d_model.y0 = [0.07446502] + [0] * (len(d_model.opt_list) - 2) + [28]
    d_model.options = 'only_FBA'  # not_add_lp_feasibility
    d_model.bound_method = 'actual_substrate_uptake'  # 'basic_Michaelis_Menten'
    d_model.growth_tol = 1E-6

    d_model.cobra_model = model.copy()
    sol_solve_ivp = dfba_functions.do_dfba_solve_ivp(d_model, bool_tqdm=False, terminal=True)
    # dfba_functions.draw(sol_solve_ivp, d_model)
    return sol_solve_ivp




### For Batch

In [3]:
from curate_kcat_light_goslim_asc_step2 import *
posterior = pickle.load(open('../Results/smc_abc_light_go_and_one_asc_step_2.pkl','rb'))

In [4]:
# prepare models with kcats from step2, but only constrain total protein content
tmp_model_file = '../Results/template_ecModel_goslim_pools.pkl'
tmp_model = pickle.load(open(tmp_model_file,'rb'))

dffrac = pd.read_csv('../Results/protein_abundance_go_slim_level_uniq_asc.csv',index_col=0)
dftot = pd.read_csv('../proteomics/total_protein_abandance_mean.csv',index_col=0)
dfpheno = pd.read_csv('../proteomics/phynotype.csv',index_col=0,comment='#')

_,pools = buildPoolDict(dffrac,'NACL60',dftot.loc['NACL60','Ptot'],sigma=0.5)

models_1 = []
sols_1 = []
for kcat_dct in posterior.population: # there are 100 kcat sets in the population
    model = tmp_model.copy()
    updateKcats(model,kcat_dct)
    updateProteinConstraints(model,pools)
    
    test_growth(model)
    models_1.append(model)
    sol_solve_ivp = do_dfba_defalut(model)
    sols_1.append(sol_solve_ivp)

file = open('sols_1.pickle', 'wb')
pickle.dump(sols_1, file)
file.close()

Using license file /Users/lhao/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file /var/folders/f_/qxjgggy53kgf53y_ytc_6nt1s_zp5x/T/tmpp9w5jjfh.lp
Reading time = 0.04 seconds
: 3871 rows, 9656 columns, 39458 nonzeros
Read LP format model from file /var/folders/f_/qxjgggy53kgf53y_ytc_6nt1s_zp5x/T/tmpen1o641y.lp
Reading time = 0.04 seconds
: 3871 rows, 9656 columns, 39458 nonzeros
Results:
  Status        : optimal
  growth_rate   : 0.15271508311216384
  glucose uptake: 0.5218257958290589
  PHA           : 0.0
  NGAM lb: 6.86
  PHA lb: 0.0

Read LP format model from file /var/folders/f_/qxjgggy53kgf53y_ytc_6nt1s_zp5x/T/tmpt6vgb6d6.lp
Reading time = 0.03 seconds
: 3871 rows, 9656 columns, 39458 nonzeros
Read LP format model from file /var/folders/f_/qxjgggy53kgf53y_ytc_6nt1s_zp5x/T/tmpmx_2xw3y.lp
Reading time = 0.04 seconds
: 3871 rows, 9656 columns, 39458 nonzeros

---------- Dynamic Flux Balance Analysis (dFBA) solve_ivp ... ---------- 
original_boun

t = 28.807: : 46it [00:09,  4.04it/s]/Users/lhao/opt/anaconda3/lib/python3.7/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)
t = 50.000: : 294it [01:02,  4.74it/s]


### For fed-batch

In [6]:
# prepare models with kcats from step2, but only constrain total protein content
tmp_model_file = '../Results/template_ecModel_goslim_pools.pkl'
tmp_model = pickle.load(open(tmp_model_file,'rb'))

dffrac = pd.read_csv('../Results/protein_abundance_go_slim_level_uniq_asc.csv',index_col=0)
dftot = pd.read_csv('../proteomics/total_protein_abandance_mean.csv',index_col=0)
dfpheno = pd.read_csv('../proteomics/phynotype.csv',index_col=0,comment='#')

Ptot = np.mean(dftot.loc[['Fermentation-9h','Fermentation-19h','Fermentation-30h'],:].values)
_,pools = buildPoolDict(dffrac,'Fermentation-19h',Ptot,sigma=0.5)

models_2 = []
sols_2 = []
for kcat_dct in posterior.population: # there are 100 kcat sets in the population
    model = tmp_model.copy()
    updateKcats(model,kcat_dct)
    updateProteinConstraints(model,pools)
    
    test_growth(model)
    models_2.append(model)
    sol_solve_ivp = do_dfba_defalut(model)
    sols_2.append(sol_solve_ivp)

file = open('sols_2.pickle', 'wb')
pickle.dump(sols_2, file)
file.close()

Read LP format model from file /var/folders/f_/qxjgggy53kgf53y_ytc_6nt1s_zp5x/T/tmp5mz4si0v.lp
Reading time = 0.08 seconds
: 3871 rows, 9656 columns, 39458 nonzeros
Read LP format model from file /var/folders/f_/qxjgggy53kgf53y_ytc_6nt1s_zp5x/T/tmpssjcfx9s.lp
Reading time = 0.04 seconds
: 3871 rows, 9656 columns, 39458 nonzeros
Results:
  Status        : optimal
  growth_rate   : 0.1242373374372529
  glucose uptake: 0.09283163541027567
  PHA           : 0.0
  NGAM lb: 6.86
  PHA lb: 0.0

Read LP format model from file /var/folders/f_/qxjgggy53kgf53y_ytc_6nt1s_zp5x/T/tmpwxko3mn6.lp
Reading time = 0.03 seconds
: 3871 rows, 9656 columns, 39458 nonzeros
Read LP format model from file /var/folders/f_/qxjgggy53kgf53y_ytc_6nt1s_zp5x/T/tmpjeeu7oqj.lp
Reading time = 0.05 seconds
: 3871 rows, 9656 columns, 39458 nonzeros

---------- Dynamic Flux Balance Analysis (dFBA) solve_ivp ... ---------- 
original_bounds: [1000.]


t = 50.000: : 112it [00:23,  4.68it/s]
